In [5]:
import importlib

import marl_env
import marl_agents
import marl_models
import marl_evaluations
importlib.reload(marl_env)
importlib.reload(marl_agents)
importlib.reload(marl_models)
importlib.reload(marl_evaluations)

from marl_env import MarelleBoard, MarelleGymEnv, MarelleGame
from marl_models import FCModel, ConvModel
from marl_agents import RandomAgent, BetterRandomAgent, SingleModelReinforce, ReinforceAgent, MarelleAgent
from marl_train import train_agent, adversarial_training
from marl_evaluations import evaluate

import progressbar as pb
import numpy as np 
import os
import cProfile

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import wandb

import torch
import torch.nn as nn

def load_model_wandb(model_name, run_id, model):
    wandb.restore(model_name, run_path=f'clement-guillo/marl/{run_id}',root= "models/temp/")
    model.load_state_dict(torch.load(f'models/temp/{model_name}'))
    os.remove(f'models/temp/{model_name}')
    return(model)

def load_model_local(model_name, model):
    model.load_state_dict(torch.load(f'models/{model_name}'))
    return(model)

env = MarelleGymEnv(end_after_place_phase=False)

# Initiate Models
einstein_conv_model_1 = ConvModel(env.N_TOTAL_ACTIONS)
einstein_conv_model_2 = ConvModel(env.N_TOTAL_ACTIONS)

# Restore models from previous trainings
# einstein_conv_model=load_model_local("3w3zmj5x_SingleModel_Conv_500epochs_1000trajectories_20201116_55%win.pt",ConvModel(env.N_TOTAL_ACTIONS))

# Agents
einstein_conv_1 = SingleModelReinforce(env=env, model=einstein_conv_model_1, lr=0.0001, win_reward=1, defeat_reward=-1, capture_reward=0, captured_reward=0, epsilon=0.1, gamma=1)
einstein_conv_2 = SingleModelReinforce(env=env, model=einstein_conv_model_2, lr=0.0001, win_reward=1, defeat_reward=-1, capture_reward=0, captured_reward=0, epsilon=0.1, gamma=1)

# Opponents
evaluate_agent=BetterRandomAgent(env)

adversarial_training(env,
    n_trajectories = 10,
    first_agent= einstein_conv_1,
    second_agent= einstein_conv_2,      
    evaluate_agent = evaluate_agent,
    log_training = False,
    save_model_freq = 999999,
    n_par_alternance = 10,
    n_dizaine_epochs = 5
)